<a href="https://colab.research.google.com/github/sankarvinayak/DL-assignment1/blob/main/DL_assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 1 DA6401 Deeplearning
## Sankar Vinayak E P

In [1]:
!pip install wandb
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cs24m041 (cs24m041-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
